# Lab2 - Expansão de Consultas
### Hadrizia Santos
Nesta atividade será exercitada a noção de expansão de consultas. Considerando a coleção de notícias do lab passado, deve-se executar os seguintes passos:

1. Escrever uma função que receba uma coleção de documentos e retorne uma matrix de termos-termos contendo as frequências de co-ocorrência de duas palavras consecutivas no texto (bigramas).
2. Escrever uma função que receba um certo termo de consulta e a matriz construída no passo 1 acima e retorneas top-3 palavras em ordem decrescente de frequencia.
3. Expandir a consulta original com os termos retornados no passo 2 acima.
4. Fazer uma busca disjuntiva (OR) considerando a nova consulta.

E responder às perguntas:

* Quais os termos retornados para a expansão de cada consulta?
* Você acha que esses termos são de fato relacionados com a consulta original? Justifique.
* Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?
* A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?

## Importar bibliotecas e os dados necessários


In [4]:
import pandas as pd
import numpy as np
from scipy import sparse
import nltk
from nltk import bigrams   
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import scipy.sparse as sps

from collections import OrderedDict

FILE_PATH = '../data/estadao_noticias_eleicao.csv'

df = pd.read_csv(FILE_PATH, encoding = 'utf-8')
df = df.replace(np.NAN, "")

# criação de uma nova coluna para a junção do título da notícia com seu conteúdo
noticia = df.titulo + ' ' + df.subTitulo + ' ' + df.conteudo

## 1. Construir matrix de ocorrência
**Obs:** O código abaixo que constrói a matriz de ocorrência foi copiado do repositório que está disponível em: https://github.com/allansales/information-retrieval/blob/master/Lab%202/coocurrence_matrix.ipynb

In [6]:
# O código abaixo está disponivel em: 
# https://github.com/allansales/information-retrieval/blob/master/Lab%202/coocurrence_matrix.ipynb

def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

def  generate_tokens(noticia):
    # Removing punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    tokens_lists = noticia.apply(lambda text: tokenizer.tokenize(text.lower()))
    
    # Removing stopwords
    stopword_ = stopwords.words('portuguese')
    filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])
    
    tokens = [token for tokens_list in filtered_tokens for token in tokens_list]
    return tokens

def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

In [7]:
tokens = generate_tokens(noticia)
matrix, vocab = co_occurrence_matrix(tokens)
consultable_matrix = matrix.tocsr()
inverted_vocab = {vocab[key]: key for key in vocab}

## 2. Retornar as top-3 palavras mais frequentes de acordo com a matriz de ocorrência
Escrever uma função que receba uma palavra do dicionário e retorne quais as 3 palavras que ocorrem mais frequentemente com o input.

In [11]:
def get_co_ocurrences(co_matrix, inv_vocab, term, N=3):
    # sparse to dense representation
    np_array = np.reshape(co_matrix[term].toarray(), -1)
    # get indice of N occurences
    return list(OrderedDict({inv_vocab[idx]: np_array[idx] for idx in (-np_array).argsort()[:N]}).keys())

## 3. Expandir a consulta original com os termos retornados no passo 2 acima.
Escrever uma função que receba a consulta, pega as 3 palavras que aparecem com maior frequência junto com esse termo e retorne uma consulta expandida.

In [16]:
def expand_query(term):
    query = []
    query.append(term)
    query.extend(get_co_ocurrences(consultable_matrix, inverted_vocab, vocab[term], 3))
    return " ".join(str(x) for x in query)

## 4. Fazer uma busca disjuntiva (OR) considerando a nova consulta.
Escrever uma função que recebe uma consulta de N termos e retorna uma consula disjuntiva entre os termos.

In [17]:
def or_search(query):
    terms = query.lower().split(' ')
    for term in terms:
        

dilma rousseff disse é
